## 12.1 RNN(Basic)

- shape(batch_size, sequence_length, input_dim) : output일 때는 input_dim은 hidden size
- 여러번 학습 시키기 위해서 batching을 이용할 수 있고 batch_size를 바꾸어 학습시킨다.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

- 한개의 시퀀스 다루기

In [3]:
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]
with tf.variable_scope('one_cell') as scope:
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

2 2
array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[-0.59105819, -0.3868193 ]]], dtype=float32)


- 여러 개의 시퀀스 다루기

In [4]:
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
array([[[ 0.1070461 ,  0.06540188],
        [ 0.34362042,  0.52931565],
        [ 0.63980752, -0.18882744],
        [ 0.69327199,  0.3343789 ],
        [-0.20043714,  0.08605383]]], dtype=float32)


## 12.2 Long Sequence RNN

In [10]:
sample = " if you want you"
idx2char = list(set(sample)) 
char2idx = {c: i for i, c in enumerate(idx2char)}

dic_size = len(char2idx) # input size
rnn_hidden_size = len(char2idx)  # output size
num_classes = len(char2idx)  # final output size
batch_size = 1
sequence_length = len(sample) - 1
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

X_one_hot = tf.one_hot(X, num_classes) 
X_for_softmax = tf.reshape(X_one_hot, [-1, rnn_hidden_size])

softmax_w = tf.get_variable("softmax_w", [rnn_hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)  # mean all sequence loss
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(3000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.60644 Prediction: uw uiyaufiyfuiy
1 loss: 2.39273 Prediction: yf yiyayy yyyiy
2 loss: 2.19401 Prediction: yf yyyayo   yyy
3 loss: 2.00903 Prediction: yf yyy yo   yyy
4 loss: 1.83703 Prediction: yf you yo   you
5 loss: 1.67824 Prediction: yf you ya   you
6 loss: 1.53255 Prediction: yf you ya   you
7 loss: 1.39932 Prediction: yf you ya   you
8 loss: 1.27751 Prediction: yf you ya t you
9 loss: 1.166 Prediction: yf you yant you
10 loss: 1.06382 Prediction: yf you yant you
11 loss: 0.970428 Prediction: yf you yant you
12 loss: 0.885575 Prediction: yf you yant you
13 loss: 0.809169 Prediction: yf you yant you
14 loss: 0.741076 Prediction: yf you yant you
15 loss: 0.681015 Prediction: yf you yant you
16 loss: 0.628527 Prediction: yf you yant you
17 loss: 0.583013 Prediction: yf you yant you
18 loss: 0.543798 Prediction: yf you yant you
19 loss: 0.510187 Prediction: yf you yant you
20 loss: 0.481506 Prediction: yf you yant you
21 loss: 0.457114 Prediction: yf you yant you
22 loss: 0.

313 loss: 0.280231 Prediction: yf you yant you
314 loss: 0.280218 Prediction: yf you yant you
315 loss: 0.280205 Prediction: yf you yant you
316 loss: 0.280192 Prediction: yf you yant you
317 loss: 0.280179 Prediction: yf you yant you
318 loss: 0.280167 Prediction: yf you yant you
319 loss: 0.280154 Prediction: yf you yant you
320 loss: 0.280141 Prediction: yf you yant you
321 loss: 0.280129 Prediction: yf you yant you
322 loss: 0.280116 Prediction: yf you yant you
323 loss: 0.280104 Prediction: yf you yant you
324 loss: 0.280092 Prediction: yf you yant you
325 loss: 0.28008 Prediction: yf you yant you
326 loss: 0.280068 Prediction: yf you yant you
327 loss: 0.280056 Prediction: yf you yant you
328 loss: 0.280044 Prediction: yf you yant you
329 loss: 0.280032 Prediction: yf you yant you
330 loss: 0.28002 Prediction: yf you yant you
331 loss: 0.280008 Prediction: yf you yant you
332 loss: 0.279997 Prediction: yf you yant you
333 loss: 0.279985 Prediction: yf you yant you
334 loss: 0.279

604 loss: 0.278401 Prediction: yf you yant you
605 loss: 0.278398 Prediction: yf you yant you
606 loss: 0.278396 Prediction: yf you yant you
607 loss: 0.278393 Prediction: yf you yant you
608 loss: 0.27839 Prediction: yf you yant you
609 loss: 0.278387 Prediction: yf you yant you
610 loss: 0.278384 Prediction: yf you yant you
611 loss: 0.278381 Prediction: yf you yant you
612 loss: 0.278379 Prediction: yf you yant you
613 loss: 0.278376 Prediction: yf you yant you
614 loss: 0.278373 Prediction: yf you yant you
615 loss: 0.27837 Prediction: yf you yant you
616 loss: 0.278368 Prediction: yf you yant you
617 loss: 0.278365 Prediction: yf you yant you
618 loss: 0.278362 Prediction: yf you yant you
619 loss: 0.278359 Prediction: yf you yant you
620 loss: 0.278357 Prediction: yf you yant you
621 loss: 0.278354 Prediction: yf you yant you
622 loss: 0.278351 Prediction: yf you yant you
623 loss: 0.278349 Prediction: yf you yant you
624 loss: 0.278346 Prediction: yf you yant you
625 loss: 0.278

864 loss: 0.277914 Prediction: yf you yant you
865 loss: 0.277913 Prediction: yf you yant you
866 loss: 0.277911 Prediction: yf you yant you
867 loss: 0.27791 Prediction: yf you yant you
868 loss: 0.277909 Prediction: yf you yant you
869 loss: 0.277908 Prediction: yf you yant you
870 loss: 0.277907 Prediction: yf you yant you
871 loss: 0.277905 Prediction: yf you yant you
872 loss: 0.277904 Prediction: yf you yant you
873 loss: 0.277903 Prediction: yf you yant you
874 loss: 0.277902 Prediction: yf you yant you
875 loss: 0.277901 Prediction: yf you yant you
876 loss: 0.2779 Prediction: yf you yant you
877 loss: 0.277898 Prediction: yf you yant you
878 loss: 0.277897 Prediction: yf you yant you
879 loss: 0.277896 Prediction: yf you yant you
880 loss: 0.277895 Prediction: yf you yant you
881 loss: 0.277894 Prediction: yf you yant you
882 loss: 0.277893 Prediction: yf you yant you
883 loss: 0.277891 Prediction: yf you yant you
884 loss: 0.27789 Prediction: yf you yant you
885 loss: 0.27788

1115 loss: 0.277692 Prediction: yf you yant you
1116 loss: 0.277691 Prediction: yf you yant you
1117 loss: 0.277691 Prediction: yf you yant you
1118 loss: 0.27769 Prediction: yf you yant you
1119 loss: 0.277689 Prediction: yf you yant you
1120 loss: 0.277689 Prediction: yf you yant you
1121 loss: 0.277688 Prediction: yf you yant you
1122 loss: 0.277687 Prediction: yf you yant you
1123 loss: 0.277687 Prediction: yf you yant you
1124 loss: 0.277686 Prediction: yf you yant you
1125 loss: 0.277686 Prediction: yf you yant you
1126 loss: 0.277685 Prediction: yf you yant you
1127 loss: 0.277684 Prediction: yf you yant you
1128 loss: 0.277684 Prediction: yf you yant you
1129 loss: 0.277683 Prediction: yf you yant you
1130 loss: 0.277682 Prediction: yf you yant you
1131 loss: 0.277682 Prediction: yf you yant you
1132 loss: 0.277681 Prediction: yf you yant you
1133 loss: 0.277681 Prediction: yf you yant you
1134 loss: 0.27768 Prediction: yf you yant you
1135 loss: 0.277679 Prediction: yf you yan

...

1685 loss: 0.277472 Prediction: yf you yant you
1686 loss: 0.277472 Prediction: yf you yant you
1687 loss: 0.277471 Prediction: yf you yant you
1688 loss: 0.277471 Prediction: yf you yant you
1689 loss: 0.277471 Prediction: yf you yant you
1690 loss: 0.277471 Prediction: yf you yant you
1691 loss: 0.277471 Prediction: yf you yant you
1692 loss: 0.27747 Prediction: yf you yant you
1693 loss: 0.27747 Prediction: yf you yant you
1694 loss: 0.27747 Prediction: yf you yant you
1695 loss: 0.27747 Prediction: yf you yant you
1696 loss: 0.277469 Prediction: yf you yant you
1697 loss: 0.277469 Prediction: yf you yant you
1698 loss: 0.277469 Prediction: yf you yant you
1699 loss: 0.277469 Prediction: yf you yant you
1700 loss: 0.277468 Prediction: yf you yant you
1701 loss: 0.277468 Prediction: yf you yant you
1702 loss: 0.277468 Prediction: yf you yant you
1703 loss: 0.277468 Prediction: yf you yant you
1704 loss: 0.277468 Prediction: yf you yant you
1705 loss: 0.277467 Prediction: yf you yant 

1920 loss: 0.277427 Prediction: yf you yant you
1921 loss: 0.277427 Prediction: yf you yant you
1922 loss: 0.277426 Prediction: yf you yant you
1923 loss: 0.277426 Prediction: yf you yant you
1924 loss: 0.277426 Prediction: yf you yant you
1925 loss: 0.277426 Prediction: yf you yant you
1926 loss: 0.277426 Prediction: yf you yant you
1927 loss: 0.277426 Prediction: yf you yant you
1928 loss: 0.277426 Prediction: yf you yant you
1929 loss: 0.277425 Prediction: yf you yant you
1930 loss: 0.277425 Prediction: yf you yant you
1931 loss: 0.277425 Prediction: yf you yant you
1932 loss: 0.277425 Prediction: yf you yant you
1933 loss: 0.277425 Prediction: yf you yant you
1934 loss: 0.277425 Prediction: yf you yant you
1935 loss: 0.277424 Prediction: yf you yant you
1936 loss: 0.277424 Prediction: yf you yant you
1937 loss: 0.277424 Prediction: yf you yant you
1938 loss: 0.277424 Prediction: yf you yant you
1939 loss: 0.277424 Prediction: yf you yant you
1940 loss: 0.277424 Prediction: yf you y

2199 loss: 0.277389 Prediction: yf you yant you
2200 loss: 0.277389 Prediction: yf you yant you
2201 loss: 0.277389 Prediction: yf you yant you
2202 loss: 0.277388 Prediction: yf you yant you
2203 loss: 0.277388 Prediction: yf you yant you
2204 loss: 0.277388 Prediction: yf you yant you
2205 loss: 0.277388 Prediction: yf you yant you
2206 loss: 0.277388 Prediction: yf you yant you
2207 loss: 0.277388 Prediction: yf you yant you
2208 loss: 0.277388 Prediction: yf you yant you
2209 loss: 0.277388 Prediction: yf you yant you
2210 loss: 0.277388 Prediction: yf you yant you
2211 loss: 0.277387 Prediction: yf you yant you
2212 loss: 0.277387 Prediction: yf you yant you
2213 loss: 0.277387 Prediction: yf you yant you
2214 loss: 0.277387 Prediction: yf you yant you
2215 loss: 0.277387 Prediction: yf you yant you
2216 loss: 0.277387 Prediction: yf you yant you
2217 loss: 0.277387 Prediction: yf you yant you
2218 loss: 0.277387 Prediction: yf you yant you
2219 loss: 0.277387 Prediction: yf you y

2436 loss: 0.277365 Prediction: yf you yant you
2437 loss: 0.277365 Prediction: yf you yant you
2438 loss: 0.277365 Prediction: yf you yant you
2439 loss: 0.277365 Prediction: yf you yant you
2440 loss: 0.277365 Prediction: yf you yant you
2441 loss: 0.277365 Prediction: yf you yant you
2442 loss: 0.277365 Prediction: yf you yant you
2443 loss: 0.277365 Prediction: yf you yant you
2444 loss: 0.277365 Prediction: yf you yant you
2445 loss: 0.277364 Prediction: yf you yant you
2446 loss: 0.277364 Prediction: yf you yant you
2447 loss: 0.277364 Prediction: yf you yant you
2448 loss: 0.277364 Prediction: yf you yant you
2449 loss: 0.277364 Prediction: yf you yant you
2450 loss: 0.277364 Prediction: yf you yant you
2451 loss: 0.277364 Prediction: yf you yant you
2452 loss: 0.277364 Prediction: yf you yant you
2453 loss: 0.277364 Prediction: yf you yant you
2454 loss: 0.277364 Prediction: yf you yant you
2455 loss: 0.277364 Prediction: yf you yant you
2456 loss: 0.277364 Prediction: yf you y

2721 loss: 0.277344 Prediction: yf you yant you
2722 loss: 0.277344 Prediction: yf you yant you
2723 loss: 0.277344 Prediction: yf you yant you
2724 loss: 0.277344 Prediction: yf you yant you
2725 loss: 0.277344 Prediction: yf you yant you
2726 loss: 0.277344 Prediction: yf you yant you
2727 loss: 0.277343 Prediction: yf you yant you
2728 loss: 0.277343 Prediction: yf you yant you
2729 loss: 0.277343 Prediction: yf you yant you
2730 loss: 0.277343 Prediction: yf you yant you
2731 loss: 0.277343 Prediction: yf you yant you
2732 loss: 0.277343 Prediction: yf you yant you
2733 loss: 0.277343 Prediction: yf you yant you
2734 loss: 0.277343 Prediction: yf you yant you
2735 loss: 0.277343 Prediction: yf you yant you
2736 loss: 0.277343 Prediction: yf you yant you
2737 loss: 0.277343 Prediction: yf you yant you
2738 loss: 0.277343 Prediction: yf you yant you
2739 loss: 0.277343 Prediction: yf you yant you
2740 loss: 0.277343 Prediction: yf you yant you
2741 loss: 0.277343 Prediction: yf you y

In [12]:
from __future__ import print_function
from tensorflow.contrib import rnn
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str] 

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)

def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)

# outputs
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

1 115            2.99586
1 116            2.99586
1 117            2.99586
1 118            2.99586
1 119            2.99586
1 120            2.99586
1 121            2.99586
1 122            2.99586
1 123            2.99586
1 124            2.99586
1 125            2.99586
1 126            2.99586
1 127            2.99586
1 128            2.99586
1 129            2.99586
1 130            2.99586
1 131            2.99586
1 132            2.99586
1 133            2.99586
1 134            2.99586
1 135            2.99586
1 136            2.99586
1 137            2.99586
1 138            2.99586
1 139            2.99586
1 140            2.99586
1 141            2.99586
1 142            2.99586
1 143            2.99586
1 144            2.99586
1 145            2.99586
1 146            2.99586
1 147            2.99586
1 148            2.99586
1 149            2.99586
1 150            2.99586
1 151            2.99586
1 152            2.99586
1 153            2.99586
1 154            2.99586


5 31            2.85915
5 32            2.85915
5 33            2.85915
5 34            2.85915
5 35            2.85915
5 36            2.85915
5 37            2.85915
5 38            2.85915
5 39            2.85915
5 40            2.85915
5 41            2.85915
5 42            2.85915
5 43            2.85915
5 44            2.85915
5 45            2.85915
5 46            2.85915
5 47            2.85915
5 48            2.85915
5 49            2.85915
5 50            2.85915
5 51            2.85915
5 52            2.85915
5 53            2.85915
5 54            2.85915
5 55            2.85915
5 56            2.85915
5 57            2.85915
5 58            2.85915
5 59            2.85915
5 60            2.85915
5 61            2.85915
5 62            2.85915
5 63            2.85915
5 64            2.85915
5 65            2.85915
5 66            2.85915
5 67            2.85915
5 68            2.85915
5 69            2.85915
5 70            2.85915
5 71            2.85915
5 72            

8 12            2.77717
8 13            2.77717
8 14            2.77717
8 15            2.77717
8 16            2.77717
8 17            2.77717
8 18            2.77717
8 19            2.77717
8 20            2.77717
8 21            2.77717
8 22            2.77717
8 23            2.77717
8 24            2.77717
8 25            2.77717
8 26            2.77717
8 27            2.77717
8 28            2.77717
8 29            2.77717
8 30            2.77717
8 31            2.77717
8 32            2.77717
8 33            2.77717
8 34            2.77717
8 35            2.77717
8 36            2.77717
8 37            2.77717
8 38            2.77717
8 39            2.77717
8 40            2.77717
8 41            2.77717
8 42            2.77717
8 43            2.77717
8 44            2.77717
8 45            2.77717
8 46            2.77717
8 47            2.77717
8 48            2.77717
8 49            2.77717
8 50            2.77717
8 51            2.77717
8 52            2.77717
8 53            

10 151     ssss   2.68254
10 152    ssss    2.68254
10 153    sss   s 2.68254
10 154   sst   ss 2.68254
10 155   st   sss 2.68254
10 156  tt    sst 2.68254
10 157   t   sst  2.68254
10 158      sst   2.68254
10 159     sst    2.68254
10 160    sst     2.68254
10 161    st      2.68254
10 162   st       2.68254
10 163   t        2.68254
10 164  t         2.68254
10 165            2.68254
10 166            2.68254
10 167            2.68254
10 168            2.68254
10 169            2.68254
11 0            2.6132
11 1            2.6132
11 2            2.6132
11 3            2.6132
11 4            2.6132
11 5     s      2.6132
11 6    s       2.6132
11 7            2.6132
11 8    s       2.6132
11 9            2.6132
11 10            2.6132
11 11            2.6132
11 12          s 2.6132
11 13         ss 2.6132
11 14        ssi 2.6132
11 15       ssis 2.6132
11 16      ssi   2.6132
11 17     ssi    2.6132
11 18   sssis    2.6132
11 19   ssi      2.6132
11 20  ssii      2.6132
11 21  ssi  

...

485 155 ns immensi 0.22879
485 156 iiimmensit 0.22879
485 157 iimmensity 0.22879
485 158 tmmensity  0.22879
485 159 mmensity o 0.22879
485 160  ensity of 0.22879
485 161  nsity of  0.22879
485 162 ndity of t 0.22879
485 163 dity of th 0.22879
485 164 ity of the 0.22879
485 165 my of the  0.22879
485 166 h of the s 0.22879
485 167  of the se 0.22879
485 168 tf the sea 0.22879
485 169 n the sea. 0.22879
486 0 f you want 0.228764
486 1 oyou want  0.228764
486 2 tou want t 0.228764
486 3  u want to 0.228764
486 4 n want to  0.228764
486 5 mwant to b 0.228764
486 6 tont to bu 0.228764
486 7 ont to bui 0.228764
486 8 nd to buil 0.228764
486 9 d to build 0.228764
486 10  to build  0.228764
486 11 to build a 0.228764
486 12   build a  0.228764
486 13 nbuild a s 0.228764
486 14 tutld a sh 0.228764
486 15 utld a shi 0.228764
486 16 mld a ship 0.228764
486 17 fd a ship, 0.228764
486 18 e a ship,  0.228764
486 19  a ship, d 0.228764
486 20 tnship, do 0.228764
486 21 nship, don 0.228764
486 22 thip

488 148 erendless  0.228751
488 149 nendless i 0.228751
488 150 tndless im 0.228751
488 151 ndless imm 0.228751
488 152 d ess imme 0.228751
488 153  ess immen 0.228751
488 154 ess immens 0.228751
488 155 ns immensi 0.228751
488 156 siimmensit 0.228751
488 157 sammensity 0.228751
488 158 tmmensity  0.228751
488 159 pmensity o 0.228751
488 160  ensity of 0.228751
488 161  nsity of  0.228751
488 162 ndity of t 0.228751
488 163 dity of th 0.228751
488 164 sgy of the 0.228751
488 165 py of the  0.228751
488 166 h of the s 0.228751
488 167  of the se 0.228751
488 168 tf the sea 0.228751
488 169   the sea. 0.228751
489 0 t you want 0.228764
489 1 oyou want  0.228764
489 2 tou want t 0.228764
489 3 ou want to 0.228764
489 4 n want to  0.228764
489 5  want to b 0.228764
489 6 tont to bu 0.228764
489 7 ont to bui 0.228764
489 8 nd to buil 0.228764
489 9 d to build 0.228764
489 10 hto build  0.228764
489 11 to luild a 0.228764
489 12 h cuild a  0.228764
489 13 nluild a s 0.228764
489 14 tutld a s

491 117  ther teac 0.22873
491 118 nher teach 0.22873
491 119  em toach  0.22873
491 120 em teach t 0.22873
491 121 m toach th 0.22873
491 122  teach the 0.22873
491 123 toach them 0.22873
491 124  ach them  0.22873
491 125 mch them t 0.22873
491 126 nh them to 0.22873
491 127 t them to  0.22873
491 128 ethem to l 0.22873
491 129 toem to lo 0.22873
491 130  em to lon 0.22873
491 131 em to long 0.22873
491 132 m to long  0.22873
491 133  ta long f 0.22873
491 134 to long fo 0.22873
491 135   long for 0.22873
491 136  long for  0.22873
491 137 tong for t 0.22873
491 138 eng for th 0.22873
491 139  ' for the 0.22873
491 140 d for the  0.22873
491 141 efor the e 0.22873
491 142 tor the en 0.22873
491 143  r the end 0.22873
491 144   the endl 0.22873
491 145  tee endle 0.22873
491 146 toemendles 0.22873
491 147  emendless 0.22873
491 148 emendless  0.22873
491 149 mtndless i 0.22873
491 150 tndless im 0.22873
491 151 msless imm 0.22873
491 152 d ess imme 0.22873
491 153  ess immen 0.22873
4

494 123 toach them 0.228745
494 124 hach them  0.228745
494 125 mch them t 0.228745
494 126 nh them to 0.228745
494 127 t them to  0.228745
494 128 ethem to l 0.228745
494 129 toem to lo 0.228745
494 130 hem to lon 0.228745
494 131 em to long 0.228745
494 132 m to long  0.228745
494 133  to long f 0.228745
494 134 to bong fo 0.228745
494 135 h long for 0.228745
494 136  long for  0.228745
494 137 tong for t 0.228745
494 138 eng for th 0.228745
494 139  ' for the 0.228745
494 140 d for the  0.228745
494 141 efor the e 0.228745
494 142 tor the en 0.228745
494 143 or the end 0.228745
494 144   the endl 0.228745
494 145  the endle 0.228745
494 146 toemendles 0.228745
494 147 hemendless 0.228745
494 148 emendless  0.228745
494 149 mtndless i 0.228745
494 150 tndless im 0.228745
494 151 msless imm 0.228745
494 152 d ess imme 0.228745
494 153  ess immen 0.228745
494 154 e s immens 0.228745
494 155 ms immensi 0.228745
494 156   immensit 0.228745
494 157  ammensity 0.228745
494 158 tmmensity  0

497 125 mch them t 0.228727
497 126 nh them to 0.228727
497 127 t them to  0.228727
497 128 ethem to l 0.228727
497 129 toem to lo 0.228727
497 130  em to lon 0.228727
497 131 em to long 0.228727
497 132 m to long  0.228727
497 133  ta long f 0.228727
497 134 to bong fo 0.228727
497 135   bong for 0.228727
497 136  long for  0.228727
497 137 tong for t 0.228727
497 138 eng for th 0.228727
497 139  ' for the 0.228727
497 140 d for the  0.228727
497 141 efor the e 0.228727
497 142 tor the en 0.228727
497 143  r the end 0.228727
497 144  kthe endl 0.228727
497 145  the endle 0.228727
497 146 toemendles 0.228727
497 147  emendless 0.228727
497 148 emendless  0.228727
497 149 mtndless i 0.228727
497 150 tndless im 0.228727
497 151 msless imm 0.228727
497 152 d ess imme 0.228727
497 153  ess immen 0.228727
497 154 e s immens 0.228727
497 155 ms immensi 0.228727
497 156  iimmensit 0.228727
497 157  immensity 0.228727
497 158 tmmensity  0.228727
497 159 pmensity o 0.228727
497 160  ensity of 0

## 12.3 Dynamic RNN

In [ ]:
x_data = np.array([[[...]]], dtype = np.float32) # 입력 데이터 지정
hidden_size = 2
cell = rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, sequence_length = [5,3,4], dtype = tf.float32)
sess.run(tf.global variables_initializer())
print(outputs.eval())

## 12.4 RNN with Time Series Data

In [16]:
import tensorflow as tf
import numpy as np
import matplotlib
import os

tf.set_random_seed(777)

if "DISPLAY" not in os.environ: 
    matplotlib.use('Agg')

import matplotlib.pyplot as plt
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

seq_length = 7
data_dim = 5
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 500

xy = np.loadtxt('data-02-stock_daily.csv', delimiter=',')
xy = xy[::-1] 

train_size = int(len(xy) * 0.7)
train_set = xy[0:train_size]
test_set = xy[train_size - seq_length:]

train_set = MinMaxScaler(train_set)
test_set = MinMaxScaler(test_set)

def build_dataset(time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(time_series) - seq_length):
        _x = time_series[i:i + seq_length, :]
        _y = time_series[i + seq_length, [-1]] 
        print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None) 

loss = tf.reduce_sum(tf.square(Y_pred - Y)) 

optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))

    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    rmse_val = sess.run(rmse, feed_dict={
                    targets: testY, predictions: test_predict})
    print("RMSE: {}".format(rmse_val))


[[  2.53065030e-01   2.45070970e-01   2.33983036e-01   4.66075110e-04
    2.32039560e-01]
 [  2.29604366e-01   2.39728936e-01   2.54567513e-01   2.98467330e-03
    2.37426028e-01]
 [  2.49235510e-01   2.41668371e-01   2.48338489e-01   2.59926504e-04
    2.26793794e-01]
 [  2.21013495e-01   2.46602231e-01   2.54710584e-01   0.00000000e+00
    2.62668239e-01]
 [  3.63433786e-01   3.70389871e-01   2.67168847e-01   1.24764722e-02
    2.62105010e-01]
 [  2.59447633e-01   3.10673724e-01   2.74113889e-01   4.56323384e-01
    2.71751265e-01]
 [  2.76008150e-01   2.78314566e-01   1.98470380e-01   5.70171193e-01
    1.78104644e-01]] -> [ 0.16053716]
[[  2.29604366e-01   2.39728936e-01   2.54567513e-01   2.98467330e-03
    2.37426028e-01]
 [  2.49235510e-01   2.41668371e-01   2.48338489e-01   2.59926504e-04
    2.26793794e-01]
 [  2.21013495e-01   2.46602231e-01   2.54710584e-01   0.00000000e+00
    2.62668239e-01]
 [  3.63433786e-01   3.70389871e-01   2.67168847e-01   1.24764722e-02
    2.621050

 [ 0.10712562  0.10171823  0.10689609  0.15204804  0.09656857]] -> [ 0.09885706]
[[ 0.04764591  0.07231955  0.06762455  0.1804607   0.06495413]
 [ 0.05554661  0.08140447  0.05956976  0.21793493  0.09216809]
 [ 0.09957004  0.11641756  0.11258831  0.17070897  0.13156305]
 [ 0.12503183  0.13642499  0.15017717  0.14748588  0.14272319]
 [ 0.13231142  0.12597895  0.13506998  0.10610379  0.12005085]
 [ 0.10712562  0.10171823  0.10689609  0.15204804  0.09656857]
 [ 0.09225582  0.08786959  0.09980797  0.13241015  0.09885706]] -> [ 0.12783116]
[[ 0.05554661  0.08140447  0.05956976  0.21793493  0.09216809]
 [ 0.09957004  0.11641756  0.11258831  0.17070897  0.13156305]
 [ 0.12503183  0.13642499  0.15017717  0.14748588  0.14272319]
 [ 0.13231142  0.12597895  0.13506998  0.10610379  0.12005085]
 [ 0.10712562  0.10171823  0.10689609  0.15204804  0.09656857]
 [ 0.09225582  0.08786959  0.09980797  0.13241015  0.09885706]
 [ 0.08642518  0.11502249  0.10747977  0.11381196  0.12783116]] -> [ 0.13103494]
[

 [ 0.55393833  0.60233661  0.59690372  0.30198978  0.59760289]] -> [ 0.59689874]
[[ 0.17977635  0.2377432   0.21174296  0.29006005  0.24132471]
 [ 0.225904    0.23996848  0.24782814  0.15924532  0.23822662]
 [ 0.2431199   0.28820734  0.27721916  0.42667384  0.30733474]
 [ 0.53251321  0.60733171  0.56361074  1.          0.6350262 ]
 [ 0.56784207  0.58831786  0.59228573  0.52460339  0.60013779]
 [ 0.55393833  0.60233661  0.59690372  0.30198978  0.59760289]
 [ 0.5735348   0.6215274   0.61372927  0.35147441  0.59689874]] -> [ 0.534163]
[[ 0.225904    0.23996848  0.24782814  0.15924532  0.23822662]
 [ 0.2431199   0.28820734  0.27721916  0.42667384  0.30733474]
 [ 0.53251321  0.60733171  0.56361074  1.          0.6350262 ]
 [ 0.56784207  0.58831786  0.59228573  0.52460339  0.60013779]
 [ 0.55393833  0.60233661  0.59690372  0.30198978  0.59760289]
 [ 0.5735348   0.6215274   0.61372927  0.35147441  0.59689874]
 [ 0.57484585  0.57045414  0.54929116  0.27078964  0.534163  ]] -> [ 0.4612174]
[[ 0

ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\tjddu\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1768, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\tjddu\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 3272, in create_op
    op_def=op_def)
  File "C:\Users\tjddu\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py", line 488, in new_func
    return func(*args, **kwargs)
